# NNとCNNの比較

In [0]:
import time
import keras
import numpy as np
import keras.backend as K
from keras.datasets import cifar10,mnist
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils

In [0]:
#precision
def P(y_true, y_pred):
    true_positives = K.sum(K.cast(K.greater(K.clip(y_true * y_pred, 0, 1), 0.20), 'float32'))
    pred_positives = K.sum(K.cast(K.greater(K.clip(y_pred, 0, 1), 0.20), 'float32'))

    precision = true_positives / (pred_positives + K.epsilon())
    return precision

#recall
def R(y_true, y_pred):
    true_positives = K.sum(K.cast(K.greater(K.clip(y_true * y_pred, 0, 1), 0.20), 'float32'))
    poss_positives = K.sum(K.cast(K.greater(K.clip(y_true, 0, 1), 0.20), 'float32'))

    recall = true_positives / (poss_positives + K.epsilon())
    return recall

In [0]:
def nn_model():
  nn_model = Sequential()
  nn_model.add(Dense(512, input_shape=x_train_nn.shape[1:]))
  nn_model.add(Activation('relu'))
  nn_model.add(Dense(10))
  nn_model.add(Activation('softmax'))
  nn_model.summary()
  nn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',P,R])
  return nn_model

In [0]:
def cnn_model():
  cnn_model = Sequential()
  cnn_model.add(Conv2D(32, (3, 3), padding="same", input_shape=(32,32,3)))
  cnn_model.add(Activation('relu'))
  cnn_model.add(Conv2D(32, (3, 3)))
  cnn_model.add(Activation('relu'))
  cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
  cnn_model.add(Dropout(0.25))
  cnn_model.add(Conv2D(64, (3, 3), padding='same'))
  cnn_model.add(Activation('relu'))
  cnn_model.add(Conv2D(64, (3, 3)))
  cnn_model.add(Activation('relu'))
  cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
  cnn_model.add(Dropout(0.25))
  cnn_model.add(Flatten())
  cnn_model.add(Dense(512))
  cnn_model.add(Activation('relu'))
  cnn_model.add(Dropout(0.5))
  cnn_model.add(Dense(10))
  cnn_model.add(Activation('softmax'))
  cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',P,R])
  return cnn_model

In [80]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)
x_train_nn = x_train.reshape(x_train.shape[0], 784)
x_test_nn = x_test.reshape(x_test.shape[0], 784)
print(x_train_nn.shape,y_train.shape,x_test_nn.shape,y_test.shape)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)
model = nn_model()
model.fit(x_train_nn, y_train, batch_size=128, epochs=5, validation_split=0.1)
score = model.evaluate(x_test_nn, y_test)
pred = model.predict(x_test_nn)
pred = np.argmax(pred, axis=1)
Y = np.argmax(y_test,axis=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print(confusion_matrix(pred,Y))

(60000, 28, 28, 1) (60000,) (10000, 28, 28, 1) (10000,)
(60000, 784) (60000,) (10000, 784) (10000,)
Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_63 (Dense)             (None, 512)               401920    
_________________________________________________________________
activation_47 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_64 (Dense)             (None, 10)                5130      
_________________________________________________________________
activation_48 (Activation)   (None, 10)                0         
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
Train on 54000 samples, validate on 6000 samples
Epoch 1/5
54000/54000 [==============================] - 7s 129us/step - loss: 12.2218 - acc: 0.